#### ID : AI28
#### Submission Date : 29th December, 2017


### Introduction

In this project, I have tried to train Machine Learning algorithm on Ames House Pricing Data. [Section 1](#S1) shows the Preprocessing of Data. [Section 2](#S2) shows how we have used 10-Fold Cross validation and selected the best architecture for the dataset. [Section 3](#S3) shows the trial of Convolutional Neural Network on the dataset.

##### Importing Libraries

In [3]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.neural_network import MLPRegressor
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_log_error,make_scorer
from sklearn.preprocessing import normalize

C:\Users\Tamim\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Tamim\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Custom Metric Function for the competition.


In [4]:

def rmsle(y_pred, y_true): 
    """
    Compute the Root Mean Squared Log Error for hypthesis y_pred and targets y_true

    Args:
        y_pred - numpy array containing predictions with shape (n_samples, n_targets)
        y_true - numpy array containing targets with shape (n_samples, n_targets)
    """
    return np.sqrt(np.square(np.log(y_pred + 1) - np.log(y_true + 1)).mean())

<a id="S1"></a>
### Section:1 Data Preprocessing


In [5]:
data = pd.read_csv('train.csv')

Replacing NaN values to zero

In [6]:
data = data.fillna(0)

Here, some columns have string values. We need to convert them to integer values

In [7]:
obj_data=data.loc[:,data.dtypes==object] 

List of Columns with String objects

In [9]:
Str_cols =obj_data.columns.tolist()
len(Str_cols)

43

In [11]:
for col in Str_cols:
    data[col] = data[col].astype('category')
    data[col] = data[col].cat.codes


In [12]:
data['Age'] = data.YrSold -data.YearBuilt

In [14]:
y =data['SalePrice']
x = data.loc[:,(data.columns !='SalePrice')&(data.columns !='Id')]

Scaling Data and normalizing the SalePrice

In [15]:
scaler = StandardScaler()
scaler.fit(x)
x = scaler.transform(x)
#x = normalize(x)
y = np.log(y)


In [16]:
x_train,x_val,y_train,y_val = train_test_split(x,y)

In [17]:
x

array([[ 0.07337496, -0.04553194,  0.2128772 , ...,  0.31386709,
         0.2085023 , -1.04325937],
       [-0.87256276, -0.04553194,  0.64574726, ...,  0.31386709,
         0.2085023 , -0.18346506],
       [ 0.07337496, -0.04553194,  0.29945121, ...,  0.31386709,
         0.2085023 , -0.97712134],
       ..., 
       [ 0.30985939, -0.04553194,  0.2417352 , ...,  0.31386709,
         0.2085023 ,  1.07315738],
       [-0.87256276, -0.04553194,  0.29945121, ...,  0.31386709,
         0.2085023 ,  0.77553627],
       [-0.87256276, -0.04553194,  0.50145724, ...,  0.31386709,
         0.2085023 ,  0.21336307]])

Reading and preprocessing Test Data

In [18]:
test_data = pd.read_csv('test.csv')

test_data = test_data.fillna(0)
test_data['Age'] = test_data.YrSold -test_data.YearBuilt
for col in Str_cols:
    test_data[col] = test_data[col].astype('category')
    test_data[col] = test_data[col].cat.codes

test_x = test_data.loc[:,(test_data.columns !='Id')]

In [19]:
test_x = scaler.transform(test_x)

In [20]:
test_x

array([[-0.87256276, -0.04553194,  0.64574726, ...,  0.95837617,
         0.2085023 ,  0.41177714],
       [-0.87256276,  1.53724502,  0.67460527, ...,  0.95837617,
         0.2085023 ,  0.51098418],
       [ 0.07337496,  1.53724502,  0.47259924, ...,  0.95837617,
         0.2085023 , -0.77870727],
       ..., 
       [-0.87256276,  1.53724502,  2.95438761, ...,  0.95837617,
        -3.42628403,  0.31257011],
       [ 0.66458604,  1.53724502,  0.12630318, ...,  0.95837617,
         0.2085023 , -0.74563826],
       [ 0.07337496,  1.53724502,  0.47259924, ...,  0.95837617,
         0.2085023 , -0.77870727]])

In [21]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Age
0,1461,20,3,80.0,11622,1,0,3,3,1,...,0,0,3,0,0,6,2010,9,4,49
1,1462,20,4,81.0,14267,1,0,0,3,1,...,0,0,0,1,12500,6,2010,9,4,52
2,1463,60,4,74.0,13830,1,0,0,3,1,...,0,0,3,0,0,3,2010,9,4,13
3,1464,60,4,78.0,9978,1,0,0,3,1,...,0,0,0,0,0,6,2010,9,4,12
4,1465,120,4,43.0,5005,1,0,0,1,1,...,0,0,0,0,0,1,2010,9,4,18


<a id="S2"></a>
### Section 2: Grid Search Cross Validation on Neural Network with Multi-Layer Perceptron

In [22]:
feature_size = x.shape[1]

In [23]:
last_layer =1
FH = (feature_size+last_layer)//2
SH = (FH+last_layer)//2
TH = (SH+last_layer)//2
FoH = (TH+last_layer)//2
FiH = (FoH+last_layer)//2
print(FH,SH,TH)

40 20 10


In [24]:
Reg = MLPRegressor(hidden_layer_sizes=(FH,SH,TH),max_iter=1000,verbose=False)
activation_options = ['identity','tanh','logistic','relu']
solver_options = ['lbfgs','adam']
alpha_options = [0.0001]
tol_options = [0.0001]
learning_rate_options = ['constant','adaptive']
param_grid_MLPReg = dict(activation = activation_options,
                       solver = solver_options,
                       alpha = alpha_options,
                       learning_rate = learning_rate_options,
                        tol=tol_options)
gridMLPReg = GridSearchCV(Reg,param_grid=param_grid_MLPReg,cv=10,scoring ='r2',verbose=2)



In [25]:
gridMLPReg.fit(x_train,y_train)

Fitting 10 folds for each of 16 candidates, totalling 160 fits
[CV] activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 
[CV]  activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   1.0s
[CV] activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 
[CV]  activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   0.1s
[CV] activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   0.1s
[CV] activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 
[CV]  activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   0.1s
[CV] activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 
[CV]  activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   0.2s
[CV] activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 
[CV]  activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   0.1s
[CV] activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 
[CV]  activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   0.1s
[CV] activation=identity, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 
[CV]  activation=identity, alpha=0.0001, learning_rate=

[CV]  activation=tanh, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   2.3s
[CV] activation=tanh, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 
[CV]  activation=tanh, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   1.6s
[CV] activation=tanh, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 
[CV]  activation=tanh, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   1.3s
[CV] activation=tanh, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 
[CV]  activation=tanh, alpha=0.0001, learning_rate=constant, solver=lbfgs, tol=0.0001 -   1.1s
[CV] activation=tanh, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 
[CV]  activation=tanh, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   3.1s
[CV] activation=tanh, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 
[CV]  activation=tanh, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   3.9s
[CV]

[CV]  activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   6.8s
[CV] activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 
[CV]  activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   7.6s
[CV] activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 
[CV]  activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   7.2s
[CV] activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 


C:\Users\Tamim\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[CV]  activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   2.5s
[CV] activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 
[CV]  activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   5.1s
[CV] activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 
[CV]  activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   3.8s
[CV] activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 
[CV]  activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   5.1s
[CV] activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 
[CV]  activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   4.0s
[CV] activation=logistic, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 
[CV]  activation=logistic, alpha=0.0001, learning_rate=constant, 

[CV]  activation=relu, alpha=0.0001, learning_rate=constant, solver=adam, tol=0.0001 -   2.6s
[CV] activation=relu, alpha=0.0001, learning_rate=adaptive, solver=lbfgs, tol=0.0001 
[CV]  activation=relu, alpha=0.0001, learning_rate=adaptive, solver=lbfgs, tol=0.0001 -   3.7s
[CV] activation=relu, alpha=0.0001, learning_rate=adaptive, solver=lbfgs, tol=0.0001 
[CV]  activation=relu, alpha=0.0001, learning_rate=adaptive, solver=lbfgs, tol=0.0001 -   3.2s
[CV] activation=relu, alpha=0.0001, learning_rate=adaptive, solver=lbfgs, tol=0.0001 
[CV]  activation=relu, alpha=0.0001, learning_rate=adaptive, solver=lbfgs, tol=0.0001 -   2.9s
[CV] activation=relu, alpha=0.0001, learning_rate=adaptive, solver=lbfgs, tol=0.0001 
[CV]  activation=relu, alpha=0.0001, learning_rate=adaptive, solver=lbfgs, tol=0.0001 -   3.1s
[CV] activation=relu, alpha=0.0001, learning_rate=adaptive, solver=lbfgs, tol=0.0001 
[CV]  activation=relu, alpha=0.0001, learning_rate=adaptive, solver=lbfgs, tol=0.0001 -   3.0s
[

[Parallel(n_jobs=1)]: Done 160 out of 160 | elapsed:  6.2min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(40, 20, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=1000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'activation': ['identity', 'tanh', 'logistic', 'relu'], 'solver': ['lbfgs', 'adam'], 'alpha': [0.0001], 'learning_rate': ['constant', 'adaptive'], 'tol': [0.0001]},
       pre_dispatch='2*n_jobs', refit=True, scoring='r2', verbose=2)

In [26]:
gridMLPReg.best_params_

{'activation': 'logistic',
 'alpha': 0.0001,
 'learning_rate': 'constant',
 'solver': 'lbfgs',
 'tol': 0.0001}

In [27]:
gridMLPReg.best_score_

0.8283036475333749

In [28]:
pred_y =gridMLPReg.predict(x_val)

In [29]:
rmsle(y_pred=pred_y,y_true=y_val)

0.013523987160705598

The best architecture

In [30]:
test_y = pd.DataFrame()
test_y['Id'] = test_data['Id']
test_y_grid = pd.DataFrame()
test_y_grid['Id'] = test_data['Id']

In [31]:
test_y_grid['SalePrice'] = np.exp(gridMLPReg.predict(test_x))

In [32]:
test_y_grid.to_csv('submission_grid.csv',index=0)

Submission Score Screenshot

![Submission Score](Submission_Grid.png)

<a id="S3"></a>
### Section 3: Convolutional Neural Network

In [17]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D, Conv1D
from keras.layers.convolutional import MaxPooling2D,MaxPooling1D
from keras.utils import np_utils
from keras.layers import Embedding
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


#### Convolutional Neural Network Function

In [70]:
def CNN(train_x_scaled_sample):
    model = Sequential()
    model.add(Conv1D(200, 1,strides=1, activation='relu', input_shape=(1,train_x_scaled_sample.shape[1])))
    model.add(Conv1D(200, 1,strides =1, activation='relu'))
    model.add(MaxPooling1D(1))
    model.add(Conv1D(100,1,strides=1, activation='relu'))
    model.add(Conv1D(100, 1,strides=1 ,activation='relu'))
    model.add(MaxPooling1D(1))
    model.add(Dropout(0.5))
    model.add(Dense(1,activation='relu'))
    return model
 

In [71]:
x_reshaped = np.reshape(x,(x.shape[0],1,x.shape[1]))
x_reshaped.shape

(1460, 1, 80)

In [72]:
y_reshape = np.reshape(y,(y.shape[0],1,-1))
y_reshape.shape

c:\users\sezan92\anaconda2\envs\py36\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  return getattr(obj, method)(*args, **kwds)


(1460, 1, 1)

In [73]:
model = CNN(x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 1, 200)            16200     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 1, 200)            40200     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 200)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 1, 100)            20100     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 1, 100)            10100     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1, 100)            0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 1, 50)             5050      
__________

In [74]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [75]:
nb_epochs = 500
model.fit(x_reshaped,y_reshape,epochs=nb_epochs,validation_split=0.1,verbose=1,batch_size=1)

Train on 1314 samples, validate on 146 samples
Epoch 1/500
1314/1314 [==============================] - 13s 10ms/step - loss: 17.3951 - val_loss: 15.7208
Epoch 2/500
1314/1314 [==============================] - 11s 9ms/step - loss: 11.0146 - val_loss: 5.2420
Epoch 3/500
1314/1314 [==============================] - 12s 9ms/step - loss: 8.3095 - val_loss: 0.7700
Epoch 4/500
1314/1314 [==============================] - 12s 9ms/step - loss: 7.4644 - val_loss: 0.7088
Epoch 5/500
1314/1314 [==============================] - 11s 9ms/step - loss: 6.5259 - val_loss: 1.1796
Epoch 6/500
1314/1314 [==============================] - 12s 9ms/step - loss: 5.6049 - val_loss: 1.5606
Epoch 7/500
1314/1314 [==============================] - 12s 9ms/step - loss: 4.9073 - val_loss: 0.5075
Epoch 8/500
1314/1314 [==============================] - 14s 11ms/step - loss: 4.6387 - val_loss: 0.5276
Epoch 9/500
1314/1314 [==============================] - 14s 11ms/step - loss: 4.6303 - val_loss: 0.6479
Epoch 10/50

1314/1314 [==============================] - 12s 9ms/step - loss: 0.0133 - val_loss: 0.0248
Epoch 156/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0145 - val_loss: 0.0197
Epoch 157/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0156 - val_loss: 0.0205
Epoch 158/500
1314/1314 [==============================] - 14s 10ms/step - loss: 0.0134 - val_loss: 0.0215
Epoch 159/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0144 - val_loss: 0.0234
Epoch 160/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0154 - val_loss: 0.0205
Epoch 161/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0179 - val_loss: 0.0208
Epoch 162/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0141 - val_loss: 0.0216
Epoch 163/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0133 - val_loss: 0.0224
Epoch 164/500
1314/1314 [==============================

1314/1314 [==============================] - 15s 11ms/step - loss: 0.0161 - val_loss: 0.0255
Epoch 233/500
1314/1314 [==============================] - 16s 12ms/step - loss: 0.0124 - val_loss: 0.0261
Epoch 234/500
1314/1314 [==============================] - 15s 11ms/step - loss: 0.0145 - val_loss: 0.0301
Epoch 235/500
1314/1314 [==============================] - 16s 13ms/step - loss: 0.0130 - val_loss: 0.0284
Epoch 236/500
1314/1314 [==============================] - 16s 12ms/step - loss: 0.0139 - val_loss: 0.0253
Epoch 237/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0142 - val_loss: 0.0263
Epoch 238/500
1314/1314 [==============================] - 14s 11ms/step - loss: 0.0156 - val_loss: 0.0252
Epoch 239/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0117 - val_loss: 0.0255
Epoch 240/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0144 - val_loss: 0.0268
Epoch 241/500
1314/1314 [============================

1314/1314 [==============================] - 12s 9ms/step - loss: 0.0127 - val_loss: 0.0229
Epoch 386/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0149 - val_loss: 0.0231
Epoch 387/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0139 - val_loss: 0.0241
Epoch 388/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0137 - val_loss: 0.0300
Epoch 389/500
1314/1314 [==============================] - 12s 10ms/step - loss: 0.0133 - val_loss: 0.0264
Epoch 390/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0130 - val_loss: 0.0263
Epoch 391/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0131 - val_loss: 0.0239
Epoch 392/500
1314/1314 [==============================] - 14s 11ms/step - loss: 0.0144 - val_loss: 0.0247
Epoch 393/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0134 - val_loss: 0.0237
Epoch 394/500
1314/1314 [=============================

Epoch 462/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0146 - val_loss: 0.0208
Epoch 463/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0145 - val_loss: 0.0266
Epoch 464/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0149 - val_loss: 0.0268
Epoch 465/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0133 - val_loss: 0.0220
Epoch 466/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0131 - val_loss: 0.0254
Epoch 467/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0147 - val_loss: 0.0209
Epoch 468/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0128 - val_loss: 0.0260
Epoch 469/500
1314/1314 [==============================] - 13s 10ms/step - loss: 0.0131 - val_loss: 0.0292
Epoch 470/500
1314/1314 [==============================] - 12s 9ms/step - loss: 0.0135 - val_loss: 0.0280
Epoch 471/500
1314/1314 [==================

In [76]:
test_x_reshape = np.reshape(test_x,(test_x.shape[0],1,test_x.shape[1]))
y_pred_cnn_reshape = model.predict(test_x_reshape)
y_pred_cnn_reshape.shape

(1459, 1, 1)

In [77]:
y_pred_cnn = np.reshape(y_pred_cnn_reshape,(-1,1))
y_pred_cnn.shape

(1459, 1)

In [78]:
y_pred_cnn = np.exp(y_pred_cnn)

In [79]:
y_pred_cnn

array([[ 127181.109375],
       [ 129366.265625],
       [ 134059.734375],
       ..., 
       [ 133711.03125 ],
       [ 120505.59375 ],
       [ 141245.3125  ]], dtype=float32)

In [80]:
test_y_cnn = pd.DataFrame()
test_y_cnn['Id'] = test_data['Id']
test_y_cnn['SalePrice'] = y_pred_cnn


In [81]:
test_y_cnn.to_csv('submission_cnn.csv',index=0)

Submission Score Screenshot

![Submission](SubmissionCNN.png)

### Conclusion

From the above experiments, we see that the above architecture of Convolutional Neural Network gives the best result of Root Mean Square Logarithmic error of 0.27315.